## Demo for Random TODOList

## refer

- https://pypi.org/project/py-trello/
- https://networkx.org/documentation/stable/tutorial.html

In [66]:
import configparser
import pandas as pd
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [67]:
from trello import TrelloClient

trello_client = TrelloClient(
    api_key=config["defaut"]["api_key"],
    api_secret=config["defaut"]["api_secret"],
    token=config["defaut"]["token"]
)
all_boards = trello_client.list_boards()
last_board = all_boards[0]
print(last_board.name)

Hi-Kanban


In [32]:
from telethon import TelegramClient, events, sync
import socks
import os

SESSION = os.environ.get('TG_SESSION', 'interactive')
API_ID = int(config["telegram"]["tg_api_id"])
API_HASH = config["telegram"]["tg_api_hash"]
PROXY = (socks.SOCKS5, 'me.niracler.com', 1080)

if f"{SESSION}.session" in os.listdir():
    os.remove(f"{SESSION}.session")

import sys
import time
client = TelegramClient(SESSION, API_ID, API_HASH, proxy=PROXY)
from IPython.display import clear_output
await client.start()
clear_output()
channel_username = 'niracler_channel'  # your channel

In [68]:
import re
category_str = "Knowledge|Output|ACG|Keep"

def map_to(x):
    weights = re.findall('\((.*?)\)[ ]*\|', x.name)
    weight = int(weights[0]) if len(weights) >= 1 else 5
    categorys = re.findall("^(" + category_str + ")[^(^|.]*", x.name)
    category = categorys[0] if len(categorys) >= 1 else "null"
    name = re.sub("^" + category + "[^|.]*\|", "", x.name).strip()
    return { 
        "name": name,
        "weight": weight, 
        "category": category, 
        "labels": [ label.name for label in x.labels ],
        "id": x.id,
        "title": x.name, 
        "short_url": x.shortUrl
    }

In [73]:
todo_list = last_board.get_list("6024ccfcd8d22a85fc633b71")
l = pd.DataFrame(list(map(map_to, todo_list.list_cards())))
l = l.set_index("id")
print(len(l))
print(l.groupby(['category'])['weight'].sum() / l['weight'].sum() * 100)
l[-10:]

59
category
ACG          32.014388
Keep          6.115108
Knowledge    48.561151
Output       13.309353
Name: weight, dtype: float64


,name,weight,category,labels,title,short_url
id,,,,,,
62ccd0aff9caa945f5e8b30c,一个月 leetcode 200 道题冲 | niracler,8,Knowledge,[],Knowledge(8) | 一个月 leetcode 200 道题冲 | niracler,https://trello.com/c/uf52V3E5
62dd189e115e4c749baf8b9b,Soul Hackers 2 | niracler,5,ACG,[],ACG | Soul Hackers 2 | niracler,https://trello.com/c/aa92MS8O
62dd190812c0ac17b3755d97,终极包管理器 Nix 调研 | niracler,10,Knowledge,[],Knowledge(10) | 终极包管理器 Nix 调研 | niracler,https://trello.com/c/Ugak6wQv
62dd197bc71a2589a9aee745,羽毛球技术提升研究 | niracler,7,Keep,[],Keep(7) | 羽毛球技术提升研究 | niracler,https://trello.com/c/cJ7MZSPe
62e87d6e49482649a15009db,画画技能培训 | niracler,5,Output,[],Output | 画画技能培训 | niracler,https://trello.com/c/GjSwPq0Y
62f6f5e8d097378a93c76648,名人传记 | Common Card | niracler,5,Knowledge,[],Knowledge | 名人传记 | Common Card | niracler,https://trello.com/c/NBVPjV38
62fa4cc0d473d0395ca7f536,《深入理解 LINUX 内核》 | niracler,5,Knowledge,[],Knowledge | 《深入理解 LINUX 内核》 | niracler,https://trello.com/c/z7jC58pn
62fa4d42295a604246bf5af9,hashicorp 系列软件深入理解 | niracler,5,Knowledge,[],Knowledge | hashicorp 系列软件深入理解 | niracler,https://trello.com/c/R0tyNUQH
62fa4d920cb7bd49be53a3a5,整理博客文章 | niracler,5,Output,[],Output | 整理博客文章 | niracler,https://trello.com/c/iIfbDdTH


In [74]:
import random, datetime, time

def rs():
    return random.choices(
        population=list(l.index),
        weights=list(l.weight),
        k=1
    )

e = datetime.datetime.now()
month = datetime.datetime.today().month

for i in range(10):
    time.sleep(3) # Delay for 3 seconds.
    random_id = rs()[0]
    if i < 9:
        print("不是: " + str(l.loc[random_id]['name']))
        
for card in todo_list.list_cards():
    if(card.id == random_id):
        card.change_list("6024cd248ecb43309b5eb2d0")
        
msg = "{month}月份就是这个了(id:{id}):\n\n{title} | {labels}\n\n恭喜恭喜!!!({time})\nurl: {url}\n#random_todolist".format(
    month=month,
    id=random_id,
    url=l.loc[random_id]['short_url'],
    title=l.loc[random_id]['title'],
    labels=l.loc[random_id]['labels'], 
    time=e.strftime("%Y-%m-%d %H:%M:%S")
)
message = await client.send_message(entity=channel_username, message=msg)
await client.pin_message(channel_username, message, notify=True)

print("\n" + msg)

不是: Rust 学习 | niracler
不是: 记录一下我看过的所有动画吧，基于myanimelist 等地毯式扫描 | 这个事情其实没有必要，关键是要对自己看过的作品负责
不是: typescript 编码规范了解一下-niracler
不是: 信息安全方面的事情了解一下 ｜ 阅读《图解密码技术》｜我已经过了一遍了，不过还是需要再深究
不是: 信息安全方面的事情了解一下 ｜ 阅读《图解密码技术》｜我已经过了一遍了，不过还是需要再深究
不是: Learning English｜《Word Power Made Easy》plan for 100 day｜100 day remaining｜All in English｜niracler
不是: 终极包管理器 Nix 调研 | niracler
不是: 最近做的工作跟系统底层打交道的情况实在是不少，需要补一下一些虚拟化方面的知识 | niracler
不是: 记录一下我看过的所有动画吧，基于myanimelist 等地毯式扫描 | 这个事情其实没有必要，关键是要对自己看过的作品负责

8月份就是这个了(id:62b57d6b0f9ab12e016e5b9f):

Output| 我想写一个维持好友关系的机器人 | niracler | []

恭喜恭喜!!!(2022-08-15 13:57:28)
url: https://trello.com/c/xH97zVRx
#random_todolist
